In [1]:
pip install cassandra-sigv4

  Using cached cassandra_sigv4-4.0.2-py2.py3-none-any.whl.metadata (5.9 kB)
  Using cached cassandra_driver-3.29.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.2 kB)
  Using cached boto3-1.38.7-py3-none-any.whl.metadata (6.6 kB)
  Using cached botocore-1.38.7-py3-none-any.whl.metadata (5.7 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached s3transfer-0.12.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached geomet-0.2.1.post1-py3-none-any.whl.metadata (1.0 kB)
Using cached cassandra_sigv4-4.0.2-py2.py3-none-any.whl (9.8 kB)
Using cached boto3-1.38.7-py3-none-any.whl (139 kB)
Using cached cassandra_driver-3.29.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.9 MB)
Using cached botocore-1.38.7-py3-none-any.whl (13.5 MB)
Using cached geomet-0.2.1.post1-py3-none-any.whl (18 kB)
Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
Using cached s3transfer-0.12.0-py3-none-any.whl (84 kB)
Note: you may need to restart the

In [2]:
!curl https://certs.secureserver.net/repository/sf-class2-root.crt -O

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1468  100  1468    0     0   6076      0 --:--:-- --:--:-- --:--:--  6091


In [3]:
from cassandra.cluster import Cluster
from ssl import SSLContext, PROTOCOL_TLSv1_2, CERT_REQUIRED
from cassandra_sigv4.auth import SigV4AuthProvider
import boto3

# ssl setup
ssl_context = SSLContext(PROTOCOL_TLSv1_2)
ssl_context.load_verify_locations('sf-class2-root.crt')  # change your file path for locating the certificate
ssl_context.verify_mode = CERT_REQUIRED

# boto3 session setup
boto_session = boto3.Session(region_name="us-east-2")  # this AWS credentials is specific to `us-east-2` region

/tmp/ipykernel_137/332746716.py:7: DeprecationWarning: ssl.PROTOCOL_TLSv1_2 is deprecated
  ssl_context = SSLContext(PROTOCOL_TLSv1_2)


In [4]:
# authorization setup with SigV4
auth_provider = SigV4AuthProvider(boto_session)

In [5]:
#cluster setup 
cluster = Cluster(['cassandra.us-east-2.amazonaws.com'], 
                  ssl_context=ssl_context, 
                  auth_provider=auth_provider, 
                  port=9142)  # TLS only communicates on port 9142

In [6]:
# establishing connection to Keyspace
session = cluster.connect()
# Insert any CQL queries between .connect() and .shutdown()

# For example, show all keyspaces created
r = session.execute('''
    SELECT * FROM system_schema.keyspaces;
    ''')
print(r.current_rows)

# For example, create a keyspace for HW2
r = session.execute('''
    CREATE KEYSPACE IF NOT EXISTS de300_demo 
    WITH replication = {'class': 'SingleRegionStrategy'};
    ''')
print(r.current_rows)

[Row(keyspace_name='system_schema', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '3')])), Row(keyspace_name='system_schema_mcs', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '3')])), Row(keyspace_name='system', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '3')])), Row(keyspace_name='system_multiregion_info', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '3')])), Row(keyspace_name='de300_axc4282', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '3')])), Row(keyspace_name='de300_demo', durable_writes=True, replication=OrderedMapSe

In [7]:
# Exercises

In [8]:
from cassandra.cluster import ExecutionProfile, EXEC_PROFILE_DEFAULT
from cassandra import ConsistencyLevel


# Define execution profile with LOCAL_QUORUM
execution_profile = ExecutionProfile(
    consistency_level=ConsistencyLevel.LOCAL_QUORUM
)

# Cluster setup with correct profile
cluster = Cluster(
    ['cassandra.us-east-2.amazonaws.com'],
    ssl_context=ssl_context,
    auth_provider=auth_provider,
    port=9142,
    execution_profiles={EXEC_PROFILE_DEFAULT: execution_profile}
)

# establishing connection to Keyspace
session = cluster.connect()
session.set_keyspace('de300_demo')  # Replace with your keyspace

In [9]:
session.execute("""
CREATE TABLE IF NOT EXISTS github (
    id UUID PRIMARY KEY,
    name TEXT,
    username TEXT
)
""")

In [10]:
import uuid

session.execute("""
    INSERT INTO github (id, name, username)
    VALUES (%s, %s, %s)
""", (uuid.uuid4(), "Your_Name", "Your_GitHub_Username"))

# Replace "Your_Name" with your name
# Replace "Your_GitHub_User_Name" with your real GitHub User

In [11]:
import csv

rows = session.execute("SELECT * FROM github")
with open("github.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["id", "name", "username"])
    for row in rows:
        writer.writerow([row.id, row.name, row.username])

In [12]:
session.shutdown()